### Нужно написать две программы:

Нужно написать две программы: 

Первая генерирует бинарный файл (min 2Гб), состоящий из случайных 32-рязрядных беззнаковых целых чисел (big endian). 

Вторая считает сумму этих чисел (с применением длинной арифметики), находит минимальное и максимальное число.

Реализуйте две версии - 
1. Простое последовательное чтение 
2. Многопоточная + memory-mapped files. 

Сравните время работы.

Import библиотек

In [1]:
import numpy as np
import time
import mmap
import os
from concurrent.futures import ThreadPoolExecutor

# Генерация файла

In [2]:
def generate_numbers(file_name, size):
    with open(file_name, 'wb') as f:
        arr = np.random.randint(2**31, 2**32-1, size=size, dtype=np.dtype('uint32').newbyteorder('B')).byteswap()
        f.write(arr.data)

In [3]:
generate_numbers('big_arr_32.txt', 550000000)

<ipython-input-2-7da200a6c0ea>:3: DeprecationWarning: Providing a dtype with a non-native byteorder is not supported. If you require platform-independent byteorder, call byteswap when required.
In future version, providing byteorder will raise a ValueError
  arr = np.random.randint(2**31, 2**32-1, size=size, dtype=np.dtype('uint32').newbyteorder('B')).byteswap()


# Простое последовательное чтение

In [4]:
def simple_solution(file_name):
    start = time.time()
    
    with open(file_name, 'r+b') as f:
        buffer = f.read()
    
    arr = np.frombuffer(buffer, dtype=np.dtype('uint32').newbyteorder('B'))
    
    arr_sum = 0
    arr_min = arr[0]
    arr_max = 0
    
    for num in arr:
        if (num > arr_max):
            arr_max = num
        if (num < arr_min):
            arr_min = num
        arr_sum += num
    
    print("Время работы ", time.time() - start)
    print("Сумма этих чисел ", arr_sum)
    print("Минимальное число ", arr_min)
    print("Максимальное число ", arr_max)

In [5]:
simple_solution('big_arr_32.txt')

Время работы  163.99773931503296
Сумма этих чисел  1771667936325433604
Минимальное число  2147483650
Максимальное число  4294967293


# Многопоточная + memory-mapped files

In [6]:
def searching(arr):
    arr_sum = 0
    arr_min = arr[0]
    arr_max = 0
    
    for num in arr:
        if (num > arr_max):
            arr_max = num
        if (num < arr_min):
            arr_min = num
        arr_sum += num
    
    return [arr_sum, arr_min, arr_max]

In [7]:
def parallel_solution(file_name):
    max_workers = 2 * os.cpu_count()
    chunk = os.path.getsize(file_name)//4//max_workers
    start = time.time()
    
    with open(file_name, 'r+b') as f:
        buffer = mmap.mmap(f.fileno(), 0, offset=0, access=mmap.ACCESS_WRITE)
        arr = np.frombuffer(buffer, dtype=np.dtype('uint32').newbyteorder('B'))

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            arrs = [arr[i*chunk:(i+1)*chunk] for i in range(max_workers)]
            future = [executor.submit(searching, ar) for ar in arrs]
        
        arr_sum = 0
        arr_min = future[0].result()[1]
        arr_max = 0
            
        for res in future:
            if (res.result()[2] > arr_max):
                arr_max = res.result()[2]
            if (res.result()[1] < arr_min):
                arr_min = res.result()[1]
            arr_sum += res.result()[0]
            
    print("Время работы ", time.time() - start)
    print("Сумма этих чисел ", arr_sum)
    print("Минимальное число ", arr_min)
    print("Максимальное число ", arr_max)

In [8]:
parallel_solution('big_arr_32.txt')

Время работы  139.0843288898468
Сумма этих чисел  1771667936325433604
Минимальное число  2147483650
Максимальное число  4294967293
